In [1]:
#we load our libraries
import pandas as pd
import sys
sys.path.append('/Users/halukamb/Documents/CodeRelated/HBDataviz_new/2025/01_BTW100Deutsche/Wrangle/lib/python3.12/site-packages')
import numpy as np
from json import loads, dumps

In [2]:
! pip install pyreadstat

In [3]:
#we have one function that filters the ALLBUS data for whether respondents are german and of voting age
def germanElectorate(df):
    german_df=df[df["german"]=="JA"]
    german_df=german_df.dropna(subset="age")
    german_df["age"]=german_df["age"].astype(float)
    german_electorate=german_df[german_df["age"]>=18]
    return german_electorate

In [8]:
#we have another function that regroups the data into fitting categories to the data we have from the zensus
def reformgroups(df):
    df["Alter"]=None
    df["Alter"]=np.where(df["age"]<30,"U30",df["Alter"])
    df["Alter"]=np.where(df["age"]>=30,"30-49",df["Alter"])
    df["Alter"]=np.where(df["age"]>=50,"50-64",df["Alter"])
    df["Alter"]=np.where(df["age"]>64,"Ü65",df["Alter"])
    df["Geschlecht"]=np.nan
    df["Geschlecht"]=np.where(df["sex"]=="FRAU","Weiblich",df["Geschlecht"])
    df["Geschlecht"]=np.where(df["sex"]=="MANN","Männlich",df["Geschlecht"])
    df["OstWest"]=np.nan
    df["OstWest"]=np.where(df["eastwest"]=="ALTE BUNDESLAENDER","West",df["OstWest"])
    df["OstWest"]=np.where(df["eastwest"]=="NEUE BUNDESLAENDER","Ost",df["OstWest"])
    df["OstWest"]=np.where(df["land"].str.contains("BERLIN"),"Berlin",df["OstWest"])

    df["Besiedelung"]=np.nan
    df["Besiedelung"]=np.where(df["gs01"].isin(['GROSSSTADT']),"dicht besiedelt",df["Besiedelung"])
    df["Besiedelung"]=np.where(df["gs01"].isin( ['MITTEL-, KLEINSTADT', 'VORORT GROSSSTADT']),"mittlere Besiedlungsdichte",df["Besiedelung"])
    df["Besiedelung"]=np.where(df["gs01"].isin( ['LAENDL. DORF','EINZELHAUS, LAND']),"gering besiedelt",df["Besiedelung"])
    
    


    
    return df

In [47]:
#and lastly we have a stratification function that takes the zensus data and the skewed dataset of allbus
#  and creates a stratified dataset based on the population shares of the zensus data
#for each combination of age, gender, eastwest and population density
def stratifier(zensus_model,skewed_df):

    stratified_dataset=pd.DataFrame()
    samplesize_sum=0

    #we go through each row of the zensus groups
    for index, row in zensus_model.iterrows():
        #take in which share of the population this group represents
        samplesize = round(row["Share"] * 100)
        samplesize_sum += samplesize
        #and then we take a copy of the skewed allbus data
        sub_df = skewed_df.copy()
        
        for i in range(0, 4):
            #and then we filter it down, feature by feature of the speficic row of the zensus data
            #we do this from 0 to 4 because in the first four rows we have the features we want to stratify by
            sub_df = sub_df[sub_df[row.index[i]] == row.values[i]]
            print(row.index[i],row.values[i],len(sub_df))
        
        
        #we draw a sample from this group that correlates to the share of the population
        #if the original dataset is too small, we sample with replacement
        sampled_subgroup = sub_df.sample(samplesize, random_state=1, replace=True)
        #and add this sampled group to the overall stratified dataset
        stratified_dataset = pd.concat([stratified_dataset, sampled_subgroup])
    print(len(stratified_dataset),samplesize_sum)
    return stratified_dataset


In [60]:
#this is the zensus data we need, we remove all rows with NaN values
#as we need as detailed data as possible for the stratification
zensus_model=pd.read_csv("zensus_model.csv")
zensus_model=zensus_model.dropna()
zensus_model=zensus_model.sort_values(by="Share",ascending=False)


In [61]:
#we read in the allbus data for 2021
df_2021 = pd.read_spss("../data/AllbusData/ZA5280_v2-0-1.sav")
# filter it for the german electorate

df_2021 = germanElectorate(df_2021)
# regroup it to fit the zensus data

df_2021_reform= reformgroups(df_2021)
# and stratify it based on the zensus data
df_2021= stratifier(zensus_model,df_2021)
df_2021

9993 9993


,za_nr,doi,version,respid,substudy,mode,splt21,eastwest,german,ep01,...,bik,gkpol,wghtpew,wghtht,wghthew,wghthtew,Alter,Geschlecht,OstWest,Besiedelung
986,ALLBUS 2021,https://doi.org/10.4232/1.14238,2.0.1 (2023-12-13),987.0,SIMULTAN,MAIL,SPLIT A,ALTE BUNDESLAENDER,JA,TEILS TEILS,...,ZONE 1 ;>499.999,100.000 - 499.999 E.,ALTE BUNDESLAENDER,1.685751,ALTE BUNDESLAENDER,2.083416,Ü65,Weiblich,West,mittlere Besiedlungsdichte
3459,ALLBUS 2021,https://doi.org/10.4232/1.14238,2.0.1 (2023-12-13),3460.0,SIMULTAN,MAIL,SPLIT C,ALTE BUNDESLAENDER,JA,NaN,...,ZONE 1 ;<500.000,100.000 - 499.999 E.,ALTE BUNDESLAENDER,NaN,NaN,NaN,Ü65,Weiblich,West,mittlere Besiedlungsdichte
1790,ALLBUS 2021,https://doi.org/10.4232/1.14238,2.0.1 (2023-12-13),1791.0,SIMULTAN,MAIL,SPLIT A,ALTE BUNDESLAENDER,JA,GUT,...,ZONE 2-4;>499.999,20.000 - 49.999 EINW,ALTE BUNDESLAENDER,0.842875,ALTE BUNDESLAENDER,1.041708,Ü65,Weiblich,West,mittlere Besiedlungsdichte
3381,ALLBUS 2021,https://doi.org/10.4232/1.14238,2.0.1 (2023-12-13),3382.0,SIMULTAN,MAIL,SPLIT A,ALTE BUNDESLAENDER,JA,NaN,...,ZONE 1 ;>499.999,500.000 UND MEHR E.,ALTE BUNDESLAENDER,1.685751,ALTE BUNDESLAENDER,2.083416,Ü65,Weiblich,West,mittlere Besiedlungsdichte
4800,ALLBUS 2021,https://doi.org/10.4232/1.14238,2.0.1 (2023-12-13),4801.0,SEQUENZIELL,CAWI,SPLIT C,ALTE BUNDESLAENDER,JA,NaN,...,ZONE 2-4;<500.000,5.000 - 19.999 EINW,ALTE BUNDESLAENDER,0.842875,ALTE BUNDESLAENDER,1.041708,Ü65,Weiblich,West,mittlere Besiedlungsdichte
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4710,ALLBUS 2021,https://doi.org/10.4232/1.14238,2.0.1 (2023-12-13),4711.0,SIMULTAN,MAIL,SPLIT A,NEUE BUNDESLAENDER,JA,TEILS TEILS,...,ZONE 2-4;<100.000,2.000 - 4.999 EINW,NEUE BUNDESLAENDER,0.267321,NEUE BUNDESLAENDER,0.141565,U30,Weiblich,Ost,gering besiedelt
1464,ALLBUS 2021,https://doi.org/10.4232/1.14238,2.0.1 (2023-12-13),1465.0,SIMULTAN,MAIL,SPLIT B,NEUE BUNDESLAENDER,JA,SCHLECHT,...,ZONE 2-4;<500.000,20.000 - 49.999 EINW,NEUE BUNDESLAENDER,0.801962,NEUE BUNDESLAENDER,0.424695,U30,Weiblich,Ost,gering besiedelt
4710,ALLBUS 2021,https://doi.org/10.4232/1.14238,2.0.1 (2023-12-13),4711.0,SIMULTAN,MAIL,SPLIT A,NEUE BUNDESLAENDER,JA,TEILS TEILS,...,ZONE 2-4;<100.000,2.000 - 4.999 EINW,NEUE BUNDESLAENDER,0.267321,NEUE BUNDESLAENDER,0.141565,U30,Weiblich,Ost,gering besiedelt
2291,ALLBUS 2021,https://doi.org/10.4232/1.14238,2.0.1 (2023-12-13),2292.0,SIMULTAN,MAIL,SPLIT A,NEUE BUNDESLAENDER,JA,TEILS TEILS,...,ZONE 2-4;>499.999,5.000 - 19.999 EINW,NEUE BUNDESLAENDER,0.801962,NEUE BUNDESLAENDER,0.424695,U30,Weiblich,Ost,gering besiedelt


In [62]:
df_2021_base = pd.read_spss("../data/AllbusData/ZA5280_v2-0-1.sav")
df_2021_base["age"]=df_2021_base["age"].astype(float)
df_2021_base[(df_2021_base["eastwest"].str.contains("NEUE"))&(df_2021_base["age"]<30)].dropna(subset=["va03"])

,za_nr,doi,version,respid,substudy,mode,splt21,eastwest,german,ep01,...,xt10c,xs15,xs16,land,bik,gkpol,wghtpew,wghtht,wghthew,wghthtew
60,ALLBUS 2021,https://doi.org/10.4232/1.14238,2.0.1 (2023-12-13),61.0,SIMULTAN,MAIL,SPLIT B,NEUE BUNDESLAENDER,JA,GUT,...,75 BIS 99 MINUTEN,80% UND MEHR GUELTIG,1 ANSCHREIBEN,SACHSEN,ZONE 1 ;>499.999,500.000 UND MEHR E.,NEUE BUNDESLAENDER,0.801962,NEUE BUNDESLAENDER,0.424695
89,ALLBUS 2021,https://doi.org/10.4232/1.14238,2.0.1 (2023-12-13),90.0,SIMULTAN,CAWI,SPLIT A,NEUE BUNDESLAENDER,JA,GUT,...,UNTER 40 MINUTEN,80% UND MEHR GUELTIG,1 ANSCHREIBEN,BRANDENBURG,ZONE 2-4;>499.999,20.000 - 49.999 EINW,NEUE BUNDESLAENDER,0.320785,NEUE BUNDESLAENDER,0.169878
95,ALLBUS 2021,https://doi.org/10.4232/1.14238,2.0.1 (2023-12-13),96.0,SEQUENZIELL,CAWI,SPLIT A,NEUE BUNDESLAENDER,JA,GUT,...,100 BIS 1440 MINUTEN,80% UND MEHR GUELTIG,2 ANSCHREIBEN,EHEM. BERLIN-OST,ZONE 1 ;>499.999,500.000 UND MEHR E.,NEUE BUNDESLAENDER,0.801962,NEUE BUNDESLAENDER,0.424695
183,ALLBUS 2021,https://doi.org/10.4232/1.14238,2.0.1 (2023-12-13),184.0,SIMULTAN,MAIL,SPLIT A,NEUE BUNDESLAENDER,JA,GUT,...,75 BIS 99 MINUTEN,80% UND MEHR GUELTIG,2 ANSCHREIBEN,SACHSEN-ANHALT,ZONE 1-4;<50.000,5.000 - 19.999 EINW,NEUE BUNDESLAENDER,1.603923,NEUE BUNDESLAENDER,0.849389
216,ALLBUS 2021,https://doi.org/10.4232/1.14238,2.0.1 (2023-12-13),217.0,SEQUENZIELL,MAIL,SPLIT B,NEUE BUNDESLAENDER,JA,TEILS TEILS,...,UNTER 40 MINUTEN,80% UND MEHR GUELTIG,3 ANSCHREIBEN,SACHSEN-ANHALT,ZONE 1 ;<500.000,100.000 - 499.999 E.,NEUE BUNDESLAENDER,0.801962,NEUE BUNDESLAENDER,0.424695
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4741,ALLBUS 2021,https://doi.org/10.4232/1.14238,2.0.1 (2023-12-13),4742.0,SIMULTAN,CAWI,SPLIT B,NEUE BUNDESLAENDER,JA,GUT,...,100 BIS 1440 MINUTEN,80% UND MEHR GUELTIG,1 ANSCHREIBEN,THUERINGEN,NaN,50.000 - 99.999 EINW,NEUE BUNDESLAENDER,1.603923,NEUE BUNDESLAENDER,0.849389
4789,ALLBUS 2021,https://doi.org/10.4232/1.14238,2.0.1 (2023-12-13),4790.0,SIMULTAN,CAWI,SPLIT B,NEUE BUNDESLAENDER,JA,GUT,...,40 BIS 59 MINUTEN,80% UND MEHR GUELTIG,3 ANSCHREIBEN,BRANDENBURG,ZONE 1-4;<50.000,20.000 - 49.999 EINW,NEUE BUNDESLAENDER,1.603923,NEUE BUNDESLAENDER,0.849389
4897,ALLBUS 2021,https://doi.org/10.4232/1.14238,2.0.1 (2023-12-13),4898.0,SIMULTAN,MAIL,SPLIT A,NEUE BUNDESLAENDER,JA,SCHLECHT,...,75 BIS 99 MINUTEN,80% UND MEHR GUELTIG,1 ANSCHREIBEN,BRANDENBURG,ZONE 2-4;>499.999,20.000 - 49.999 EINW,NEUE BUNDESLAENDER,0.801962,NEUE BUNDESLAENDER,0.424695
4930,ALLBUS 2021,https://doi.org/10.4232/1.14238,2.0.1 (2023-12-13),4931.0,SIMULTAN,CAWI,SPLIT B,NEUE BUNDESLAENDER,JA,GUT,...,60 BIS 74 MINUTEN,80% UND MEHR GUELTIG,1 ANSCHREIBEN,SACHSEN-ANHALT,ZONE 1 ;<100.000,20.000 - 49.999 EINW,NEUE BUNDESLAENDER,1.603923,NEUE BUNDESLAENDER,0.849389


In [63]:

df_2021_reform["age"]=df_2021_reform["age"].astype(float)
df_2021_reform[(df_2021_reform["eastwest"].str.contains("NEUE"))&(df_2021_reform["age"]<30)].dropna(subset=["va03"])

,za_nr,doi,version,respid,substudy,mode,splt21,eastwest,german,ep01,...,bik,gkpol,wghtpew,wghtht,wghthew,wghthtew,Alter,Geschlecht,OstWest,Besiedelung
60,ALLBUS 2021,https://doi.org/10.4232/1.14238,2.0.1 (2023-12-13),61.0,SIMULTAN,MAIL,SPLIT B,NEUE BUNDESLAENDER,JA,GUT,...,ZONE 1 ;>499.999,500.000 UND MEHR E.,NEUE BUNDESLAENDER,0.801962,NEUE BUNDESLAENDER,0.424695,U30,Männlich,Ost,dicht besiedelt
89,ALLBUS 2021,https://doi.org/10.4232/1.14238,2.0.1 (2023-12-13),90.0,SIMULTAN,CAWI,SPLIT A,NEUE BUNDESLAENDER,JA,GUT,...,ZONE 2-4;>499.999,20.000 - 49.999 EINW,NEUE BUNDESLAENDER,0.320785,NEUE BUNDESLAENDER,0.169878,U30,Männlich,Ost,mittlere Besiedlungsdichte
95,ALLBUS 2021,https://doi.org/10.4232/1.14238,2.0.1 (2023-12-13),96.0,SEQUENZIELL,CAWI,SPLIT A,NEUE BUNDESLAENDER,JA,GUT,...,ZONE 1 ;>499.999,500.000 UND MEHR E.,NEUE BUNDESLAENDER,0.801962,NEUE BUNDESLAENDER,0.424695,U30,Weiblich,Berlin,dicht besiedelt
183,ALLBUS 2021,https://doi.org/10.4232/1.14238,2.0.1 (2023-12-13),184.0,SIMULTAN,MAIL,SPLIT A,NEUE BUNDESLAENDER,JA,GUT,...,ZONE 1-4;<50.000,5.000 - 19.999 EINW,NEUE BUNDESLAENDER,1.603923,NEUE BUNDESLAENDER,0.849389,U30,Weiblich,Ost,gering besiedelt
216,ALLBUS 2021,https://doi.org/10.4232/1.14238,2.0.1 (2023-12-13),217.0,SEQUENZIELL,MAIL,SPLIT B,NEUE BUNDESLAENDER,JA,TEILS TEILS,...,ZONE 1 ;<500.000,100.000 - 499.999 E.,NEUE BUNDESLAENDER,0.801962,NEUE BUNDESLAENDER,0.424695,U30,Männlich,Ost,dicht besiedelt
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4741,ALLBUS 2021,https://doi.org/10.4232/1.14238,2.0.1 (2023-12-13),4742.0,SIMULTAN,CAWI,SPLIT B,NEUE BUNDESLAENDER,JA,GUT,...,NaN,50.000 - 99.999 EINW,NEUE BUNDESLAENDER,1.603923,NEUE BUNDESLAENDER,0.849389,U30,Weiblich,Ost,dicht besiedelt
4789,ALLBUS 2021,https://doi.org/10.4232/1.14238,2.0.1 (2023-12-13),4790.0,SIMULTAN,CAWI,SPLIT B,NEUE BUNDESLAENDER,JA,GUT,...,ZONE 1-4;<50.000,20.000 - 49.999 EINW,NEUE BUNDESLAENDER,1.603923,NEUE BUNDESLAENDER,0.849389,U30,Männlich,Ost,mittlere Besiedlungsdichte
4897,ALLBUS 2021,https://doi.org/10.4232/1.14238,2.0.1 (2023-12-13),4898.0,SIMULTAN,MAIL,SPLIT A,NEUE BUNDESLAENDER,JA,SCHLECHT,...,ZONE 2-4;>499.999,20.000 - 49.999 EINW,NEUE BUNDESLAENDER,0.801962,NEUE BUNDESLAENDER,0.424695,U30,Weiblich,Ost,mittlere Besiedlungsdichte
4930,ALLBUS 2021,https://doi.org/10.4232/1.14238,2.0.1 (2023-12-13),4931.0,SIMULTAN,CAWI,SPLIT B,NEUE BUNDESLAENDER,JA,GUT,...,ZONE 1 ;<100.000,20.000 - 49.999 EINW,NEUE BUNDESLAENDER,1.603923,NEUE BUNDESLAENDER,0.849389,U30,Männlich,Ost,mittlere Besiedlungsdichte


In [64]:

df_2021["age"]=df_2021["age"].astype(float)
df_2021[(df_2021["eastwest"].str.contains("NEUE"))&(df_2021["age"]<30)].dropna(subset=["va03"])

,za_nr,doi,version,respid,substudy,mode,splt21,eastwest,german,ep01,...,bik,gkpol,wghtpew,wghtht,wghthew,wghthtew,Alter,Geschlecht,OstWest,Besiedelung
1068,ALLBUS 2021,https://doi.org/10.4232/1.14238,2.0.1 (2023-12-13),1069.0,SIMULTAN,MAIL,SPLIT A,NEUE BUNDESLAENDER,JA,TEILS TEILS,...,ZONE 1-4;<50.000,5.000 - 19.999 EINW,NEUE BUNDESLAENDER,1.603923,NEUE BUNDESLAENDER,0.849389,U30,Männlich,Ost,mittlere Besiedlungsdichte
2241,ALLBUS 2021,https://doi.org/10.4232/1.14238,2.0.1 (2023-12-13),2242.0,SIMULTAN,MAIL,SPLIT A,NEUE BUNDESLAENDER,JA,GUT,...,ZONE 1 ;<500.000,50.000 - 99.999 EINW,NEUE BUNDESLAENDER,0.801962,NEUE BUNDESLAENDER,0.424695,U30,Männlich,Ost,mittlere Besiedlungsdichte
2314,ALLBUS 2021,https://doi.org/10.4232/1.14238,2.0.1 (2023-12-13),2315.0,SIMULTAN,CAWI,SPLIT A,NEUE BUNDESLAENDER,JA,TEILS TEILS,...,ZONE 1 ;<500.000,50.000 - 99.999 EINW,NEUE BUNDESLAENDER,1.603923,NEUE BUNDESLAENDER,0.849389,U30,Männlich,Ost,mittlere Besiedlungsdichte
1466,ALLBUS 2021,https://doi.org/10.4232/1.14238,2.0.1 (2023-12-13),1467.0,SIMULTAN,CAWI,SPLIT A,NEUE BUNDESLAENDER,JA,GUT,...,ZONE 2-4;<100.000,5.000 - 19.999 EINW,NEUE BUNDESLAENDER,1.603923,NEUE BUNDESLAENDER,0.849389,U30,Männlich,Ost,mittlere Besiedlungsdichte
1857,ALLBUS 2021,https://doi.org/10.4232/1.14238,2.0.1 (2023-12-13),1858.0,SEQUENZIELL,CAWI,SPLIT B,NEUE BUNDESLAENDER,JA,SCHLECHT,...,ZONE 2-4;<100.000,5.000 - 19.999 EINW,NEUE BUNDESLAENDER,0.801962,NEUE BUNDESLAENDER,0.424695,U30,Männlich,Ost,mittlere Besiedlungsdichte
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4710,ALLBUS 2021,https://doi.org/10.4232/1.14238,2.0.1 (2023-12-13),4711.0,SIMULTAN,MAIL,SPLIT A,NEUE BUNDESLAENDER,JA,TEILS TEILS,...,ZONE 2-4;<100.000,2.000 - 4.999 EINW,NEUE BUNDESLAENDER,0.267321,NEUE BUNDESLAENDER,0.141565,U30,Weiblich,Ost,gering besiedelt
1464,ALLBUS 2021,https://doi.org/10.4232/1.14238,2.0.1 (2023-12-13),1465.0,SIMULTAN,MAIL,SPLIT B,NEUE BUNDESLAENDER,JA,SCHLECHT,...,ZONE 2-4;<500.000,20.000 - 49.999 EINW,NEUE BUNDESLAENDER,0.801962,NEUE BUNDESLAENDER,0.424695,U30,Weiblich,Ost,gering besiedelt
4710,ALLBUS 2021,https://doi.org/10.4232/1.14238,2.0.1 (2023-12-13),4711.0,SIMULTAN,MAIL,SPLIT A,NEUE BUNDESLAENDER,JA,TEILS TEILS,...,ZONE 2-4;<100.000,2.000 - 4.999 EINW,NEUE BUNDESLAENDER,0.267321,NEUE BUNDESLAENDER,0.141565,U30,Weiblich,Ost,gering besiedelt
2291,ALLBUS 2021,https://doi.org/10.4232/1.14238,2.0.1 (2023-12-13),2292.0,SIMULTAN,MAIL,SPLIT A,NEUE BUNDESLAENDER,JA,TEILS TEILS,...,ZONE 2-4;>499.999,5.000 - 19.999 EINW,NEUE BUNDESLAENDER,0.801962,NEUE BUNDESLAENDER,0.424695,U30,Weiblich,Ost,gering besiedelt


In [65]:
df_2021["OstWest"].unique()

array(['West', 'Ost', 'Berlin'], dtype=object)

In [66]:
#we read in the allbus data for 2023
df_2023 = pd.read_spss("../data/AllbusData/ZA8830_v1-0-0.sav")
# filter it for the german electorate

df_2023 = germanElectorate(df_2023)
# regroup it to fit the zensus data

df_2023= reformgroups(df_2023)
#and straitify it based on the zensus data
df_2023= stratifier(zensus_model,df_2023)
df_2023


9993 9993


,za_nr,doi,version,respid,substudy,mode,splt23_1,splt23_2,eastwest,german,...,bik,xs11,wghtpew,wghtht,wghthew,wghthtew,Alter,Geschlecht,OstWest,Besiedelung
770,ALLBUS 2023,https://doi.org/10.4232/1.14387,"v1.0.0, 2024-12-03",771.0,NaN,CAPI,NaN,SPLIT A,ALTE BUNDESLAENDER,JA,...,ZONE 2-4;>499.999,183.0,1.252326,1.69639,1.242308,2.107439,Ü65,Weiblich,West,mittlere Besiedlungsdichte
3538,ALLBUS 2023,https://doi.org/10.4232/1.14387,"v1.0.0, 2024-12-03",3539.0,NaN,CAPI,NaN,SPLIT B,ALTE BUNDESLAENDER,JA,...,ZONE 1 ;>499.999,93.0,1.252326,0.848195,1.242308,1.053719,Ü65,Weiblich,West,mittlere Besiedlungsdichte
1776,ALLBUS 2023,https://doi.org/10.4232/1.14387,"v1.0.0, 2024-12-03",1777.0,SIMULTAN,MAIL,SPLIT B,TNZ: MODE,ALTE BUNDESLAENDER,JA,...,ZONE 1-4;<50.000,10.0,1.203727,1.693259,1.192166,2.018646,Ü65,Weiblich,West,mittlere Besiedlungsdichte
3432,ALLBUS 2023,https://doi.org/10.4232/1.14387,"v1.0.0, 2024-12-03",3433.0,SEQUENZIELL,CAWI,SPLIT A,TNZ: MODE,ALTE BUNDESLAENDER,JA,...,ZONE 1 ;<500.000,168.0,1.207467,0.554394,1.205184,0.668147,Ü65,Weiblich,West,mittlere Besiedlungsdichte
5150,ALLBUS 2023,https://doi.org/10.4232/1.14387,"v1.0.0, 2024-12-03",5151.0,NaN,CAPI,NaN,SPLIT B,ALTE BUNDESLAENDER,JA,...,ZONE 2-4;>499.999,143.0,1.252326,0.848195,1.242308,1.053719,Ü65,Weiblich,West,mittlere Besiedlungsdichte
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1897,ALLBUS 2023,https://doi.org/10.4232/1.14387,"v1.0.0, 2024-12-03",1898.0,NaN,CAPI,NaN,SPLIT B,NEUE BUNDESLAENDER,JA,...,5.000-19.999 EINW.,55.0,0.501346,0.532744,0.524886,0.27963,U30,Weiblich,Ost,gering besiedelt
3272,ALLBUS 2023,https://doi.org/10.4232/1.14387,"v1.0.0, 2024-12-03",3273.0,SIMULTAN,MAIL,SPLIT A,TNZ: MODE,NEUE BUNDESLAENDER,JA,...,ZONE 1 ;<500.000,176.0,0.544816,0.845091,0.572064,0.483446,U30,Weiblich,Ost,gering besiedelt
4876,ALLBUS 2023,https://doi.org/10.4232/1.14387,"v1.0.0, 2024-12-03",4877.0,SIMULTAN,MAIL,SPLIT A,TNZ: MODE,NEUE BUNDESLAENDER,JA,...,ZONE 2-4;<500.000,31.0,0.544816,0.338036,0.572064,0.193378,U30,Weiblich,Ost,gering besiedelt
4876,ALLBUS 2023,https://doi.org/10.4232/1.14387,"v1.0.0, 2024-12-03",4877.0,SIMULTAN,MAIL,SPLIT A,TNZ: MODE,NEUE BUNDESLAENDER,JA,...,ZONE 2-4;<500.000,31.0,0.544816,0.338036,0.572064,0.193378,U30,Weiblich,Ost,gering besiedelt


In [67]:
#this is a function that enables us to analyze for each for each subgroup how the respondents answered differently
#  to a given question/variable
def analyze_factor_patterns(dataset,variable_to_check):
    #we have a base dataframe that holds all results
    base_df=pd.DataFrame()
    #now we define the four features we want to use to check for changes in the respondents data in allbus
    factor_list=["Alter","Geschlecht","OstWest","Besiedelung"]
    #we iterate through all possible combinations of these features
    for element in factor_list:
        #print(element)
        for element2 in factor_list:
            #add we za_nr to the list of the variable to check, as it is the unique identifier for each respondent
            baselist=[variable_to_check,"za_nr"]

            #we build also a list of the features we want to check for
            extralist=[element,element2]
            #set it to remove duplicates, in case we pick the same feature twice
            extralist=list(set(extralist))

            #and combine the two lists and filter the dataset for the columns we need
            baselist.extend(extralist)
            dataset_reduced=dataset[baselist]
            #remove all rows with missing values
            dataset_reduced = dataset_reduced[~dataset_reduced.isin(["nan"]).any(axis=1)]

            #we also define a extralist plus which we need to group by the dataset
            #it is the name of the features we want to check for plus the variable
            # and then we count
            extralist_plus_factor=extralist+[variable_to_check]
            grouped_df=dataset_reduced.groupby(by=extralist_plus_factor).count()
            #we remove all columns that have missing values
            grouped_df=grouped_df.dropna(axis=1)
            #and then we calculate the share of the respondents for each subgroup by using the sum of the group
            grouped_df['percent'] = grouped_df['za_nr']*100 / grouped_df.groupby(by=extralist)['za_nr'].transform('sum')
            grouped_df['absolute'] = grouped_df['za_nr']
            
            #and we add this to the base dataframe that holds all results
            grouped_df=grouped_df.reset_index()
            base_df=pd.concat([base_df,grouped_df],axis=0)
    #at the end we drop duplicates
    base_df=base_df.drop_duplicates()
    #reduce the dataset
    base_df_cols=['Alter', 'Besiedelung', 'Geschlecht', 'OstWest', variable_to_check,
       'absolute','percent']
    #and return the dataset
    return base_df[base_df_cols]

In [68]:
# now we do this whole procedure for variable va03 which is for inflation
df_2023_inflation=analyze_factor_patterns(df_2023,"va03")
df_2023_inflation.sort_values(by="absolute",ascending=False)

/var/folders/m2/_syds9sn2bjctwpf6p3_w19r0000gn/T/ipykernel_38524/1110048911.py:30: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_df=dataset_reduced.groupby(by=extralist_plus_factor).count()
/var/folders/m2/_syds9sn2bjctwpf6p3_w19r0000gn/T/ipykernel_38524/1110048911.py:30: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_df=dataset_reduced.groupby(by=extralist_plus_factor).count()
/var/folders/m2/_syds9sn2bjctwpf6p3_w19r0000gn/T/ipykernel_38524/1110048911.py:30: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False

,Alter,Besiedelung,Geschlecht,OstWest,va03,absolute,percent
9,NaN,NaN,NaN,West,AM VIERTWICHTIGSTEN,3075,39.738951
8,NaN,NaN,NaN,West,AM DRITTWICHTIGSTEN,2145,27.720341
1,NaN,NaN,Männlich,NaN,AM VIERTWICHTIGSTEN,1958,41.325454
5,NaN,NaN,Weiblich,NaN,AM VIERTWICHTIGSTEN,1900,37.984806
9,NaN,mittlere Besiedlungsdichte,NaN,NaN,AM VIERTWICHTIGSTEN,1658,38.675064
...,...,...,...,...,...,...,...
7,NaN,gering besiedelt,NaN,Berlin,AM ZWEITWICHTIGSTEN,0,NaN
8,NaN,mittlere Besiedlungsdichte,NaN,Berlin,AM DRITTWICHTIGSTEN,0,NaN
9,NaN,mittlere Besiedlungsdichte,NaN,Berlin,AM VIERTWICHTIGSTEN,0,NaN
10,NaN,mittlere Besiedlungsdichte,NaN,Berlin,AM WICHTIGSTEN,0,NaN


In [69]:
df_2023_inflation[df_2023_inflation.percent.isna()]

,Alter,Besiedelung,Geschlecht,OstWest,va03,absolute,percent
4,NaN,gering besiedelt,NaN,Berlin,AM DRITTWICHTIGSTEN,0,NaN
5,NaN,gering besiedelt,NaN,Berlin,AM VIERTWICHTIGSTEN,0,NaN
6,NaN,gering besiedelt,NaN,Berlin,AM WICHTIGSTEN,0,NaN
7,NaN,gering besiedelt,NaN,Berlin,AM ZWEITWICHTIGSTEN,0,NaN
8,NaN,mittlere Besiedlungsdichte,NaN,Berlin,AM DRITTWICHTIGSTEN,0,NaN
9,NaN,mittlere Besiedlungsdichte,NaN,Berlin,AM VIERTWICHTIGSTEN,0,NaN
10,NaN,mittlere Besiedlungsdichte,NaN,Berlin,AM WICHTIGSTEN,0,NaN
11,NaN,mittlere Besiedlungsdichte,NaN,Berlin,AM ZWEITWICHTIGSTEN,0,NaN


In [70]:
df_2023_inflation[["Alter","Besiedelung","Geschlecht","OstWest","absolute"]]

,Alter,Besiedelung,Geschlecht,OstWest,absolute
0,30-49,NaN,NaN,NaN,714
1,30-49,NaN,NaN,NaN,925
2,30-49,NaN,NaN,NaN,426
3,30-49,NaN,NaN,NaN,684
4,50-64,NaN,NaN,NaN,784
...,...,...,...,...,...
7,NaN,gering besiedelt,NaN,NaN,486
8,NaN,mittlere Besiedlungsdichte,NaN,NaN,1153
9,NaN,mittlere Besiedlungsdichte,NaN,NaN,1658
10,NaN,mittlere Besiedlungsdichte,NaN,NaN,501


In [71]:
# now we do this whole procedure for variable va03 which is for inflation

df_2021_inflation=analyze_factor_patterns(df_2021,"va03")
df_2021_inflation.sort_values(by="absolute",ascending=False)

/var/folders/m2/_syds9sn2bjctwpf6p3_w19r0000gn/T/ipykernel_38524/1110048911.py:30: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_df=dataset_reduced.groupby(by=extralist_plus_factor).count()
/var/folders/m2/_syds9sn2bjctwpf6p3_w19r0000gn/T/ipykernel_38524/1110048911.py:30: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_df=dataset_reduced.groupby(by=extralist_plus_factor).count()
/var/folders/m2/_syds9sn2bjctwpf6p3_w19r0000gn/T/ipykernel_38524/1110048911.py:30: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False

,Alter,Besiedelung,Geschlecht,OstWest,va03,absolute,percent
9,NaN,NaN,NaN,West,AM VIERTWICHTIGSTEN,2781,55.675676
1,NaN,NaN,Männlich,NaN,AM VIERTWICHTIGSTEN,1801,56.760164
5,NaN,NaN,Weiblich,NaN,AM VIERTWICHTIGSTEN,1671,52.431754
9,NaN,mittlere Besiedlungsdichte,NaN,NaN,AM VIERTWICHTIGSTEN,1573,56.481149
17,NaN,NaN,Männlich,West,AM VIERTWICHTIGSTEN,1458,57.788347
...,...,...,...,...,...,...,...
7,NaN,gering besiedelt,NaN,Berlin,AM ZWEITWICHTIGSTEN,0,NaN
8,NaN,mittlere Besiedlungsdichte,NaN,Berlin,AM DRITTWICHTIGSTEN,0,NaN
9,NaN,mittlere Besiedlungsdichte,NaN,Berlin,AM VIERTWICHTIGSTEN,0,NaN
10,NaN,mittlere Besiedlungsdichte,NaN,Berlin,AM WICHTIGSTEN,0,NaN


In [72]:
df_2021_inflation[df_2021_inflation.percent.isna()]

,Alter,Besiedelung,Geschlecht,OstWest,va03,absolute,percent
4,NaN,gering besiedelt,NaN,Berlin,AM DRITTWICHTIGSTEN,0,NaN
5,NaN,gering besiedelt,NaN,Berlin,AM VIERTWICHTIGSTEN,0,NaN
6,NaN,gering besiedelt,NaN,Berlin,AM WICHTIGSTEN,0,NaN
7,NaN,gering besiedelt,NaN,Berlin,AM ZWEITWICHTIGSTEN,0,NaN
8,NaN,mittlere Besiedlungsdichte,NaN,Berlin,AM DRITTWICHTIGSTEN,0,NaN
9,NaN,mittlere Besiedlungsdichte,NaN,Berlin,AM VIERTWICHTIGSTEN,0,NaN
10,NaN,mittlere Besiedlungsdichte,NaN,Berlin,AM WICHTIGSTEN,0,NaN
11,NaN,mittlere Besiedlungsdichte,NaN,Berlin,AM ZWEITWICHTIGSTEN,0,NaN


In [92]:
df_2023_inflation[df_2023_inflation.percent.isna()]

,Alter,Besiedelung,Geschlecht,OstWest,va03,absolute,percent
4,NaN,gering besiedelt,NaN,Berlin,AM DRITTWICHTIGSTEN,0,NaN
5,NaN,gering besiedelt,NaN,Berlin,AM VIERTWICHTIGSTEN,0,NaN
6,NaN,gering besiedelt,NaN,Berlin,AM WICHTIGSTEN,0,NaN
7,NaN,gering besiedelt,NaN,Berlin,AM ZWEITWICHTIGSTEN,0,NaN
8,NaN,mittlere Besiedlungsdichte,NaN,Berlin,AM DRITTWICHTIGSTEN,0,NaN
9,NaN,mittlere Besiedlungsdichte,NaN,Berlin,AM VIERTWICHTIGSTEN,0,NaN
10,NaN,mittlere Besiedlungsdichte,NaN,Berlin,AM WICHTIGSTEN,0,NaN
11,NaN,mittlere Besiedlungsdichte,NaN,Berlin,AM ZWEITWICHTIGSTEN,0,NaN


In [73]:
# and we merge the datasets of 2021 and 2023 based on all the features
comp_inflation=pd.merge(df_2021_inflation,df_2023_inflation,on=['Alter', 'Besiedelung', 'Geschlecht', 'OstWest', 'va03'],suffixes=["_2021","_2023"])
comp_inflation

,Alter,Besiedelung,Geschlecht,OstWest,va03,absolute_2021,percent_2021,absolute_2023,percent_2023
0,30-49,NaN,NaN,NaN,AM DRITTWICHTIGSTEN,541,29.872998,714,25.973081
1,30-49,NaN,NaN,NaN,AM VIERTWICHTIGSTEN,843,46.548868,925,33.648599
2,30-49,NaN,NaN,NaN,AM WICHTIGSTEN,155,8.558807,426,15.496544
3,30-49,NaN,NaN,NaN,AM ZWEITWICHTIGSTEN,272,15.019326,684,24.881775
4,50-64,NaN,NaN,NaN,AM DRITTWICHTIGSTEN,418,22.668113,784,28.040057
...,...,...,...,...,...,...,...,...,...
255,NaN,gering besiedelt,NaN,NaN,AM ZWEITWICHTIGSTEN,226,16.715976,486,23.800196
256,NaN,mittlere Besiedlungsdichte,NaN,NaN,AM DRITTWICHTIGSTEN,727,26.104129,1153,26.895265
257,NaN,mittlere Besiedlungsdichte,NaN,NaN,AM VIERTWICHTIGSTEN,1573,56.481149,1658,38.675064
258,NaN,mittlere Besiedlungsdichte,NaN,NaN,AM WICHTIGSTEN,115,4.129264,501,11.686494


In [74]:
#also we do the same for party preferences
df_2021_party=analyze_factor_patterns(df_2021,"pv01")
df_2021_party

/var/folders/m2/_syds9sn2bjctwpf6p3_w19r0000gn/T/ipykernel_38524/1110048911.py:30: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_df=dataset_reduced.groupby(by=extralist_plus_factor).count()
/var/folders/m2/_syds9sn2bjctwpf6p3_w19r0000gn/T/ipykernel_38524/1110048911.py:30: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_df=dataset_reduced.groupby(by=extralist_plus_factor).count()
/var/folders/m2/_syds9sn2bjctwpf6p3_w19r0000gn/T/ipykernel_38524/1110048911.py:30: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False

,Alter,Besiedelung,Geschlecht,OstWest,pv01,absolute,percent
0,30-49,NaN,NaN,NaN,AFD,137,6.135244
1,30-49,NaN,NaN,NaN,ANDERE PARTEI,114,5.105240
2,30-49,NaN,NaN,NaN,CDU-CSU,477,21.361397
3,30-49,NaN,NaN,NaN,DIE GRUENEN,743,33.273623
4,30-49,NaN,NaN,NaN,DIE LINKE,143,6.403941
...,...,...,...,...,...,...,...
19,NaN,mittlere Besiedlungsdichte,NaN,NaN,DIE GRUENEN,881,25.403691
20,NaN,mittlere Besiedlungsdichte,NaN,NaN,DIE LINKE,200,5.767013
21,NaN,mittlere Besiedlungsdichte,NaN,NaN,FDP,471,13.581315
22,NaN,mittlere Besiedlungsdichte,NaN,NaN,SPD,537,15.484429


In [75]:
df_2023_party=analyze_factor_patterns(df_2023,"pv01")
df_2023_party

/var/folders/m2/_syds9sn2bjctwpf6p3_w19r0000gn/T/ipykernel_38524/1110048911.py:30: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_df=dataset_reduced.groupby(by=extralist_plus_factor).count()
/var/folders/m2/_syds9sn2bjctwpf6p3_w19r0000gn/T/ipykernel_38524/1110048911.py:30: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_df=dataset_reduced.groupby(by=extralist_plus_factor).count()
/var/folders/m2/_syds9sn2bjctwpf6p3_w19r0000gn/T/ipykernel_38524/1110048911.py:30: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False

,Alter,Besiedelung,Geschlecht,OstWest,pv01,absolute,percent
0,30-49,NaN,NaN,NaN,AFD,271,12.212708
1,30-49,NaN,NaN,NaN,ANDERE PARTEI,77,3.470032
2,30-49,NaN,NaN,NaN,CDU-CSU,426,19.197837
3,30-49,NaN,NaN,NaN,DIE GRUENEN,534,24.064894
4,30-49,NaN,NaN,NaN,DIE LINKE,138,6.219018
...,...,...,...,...,...,...,...
19,NaN,mittlere Besiedlungsdichte,NaN,NaN,DIE GRUENEN,711,19.921547
20,NaN,mittlere Besiedlungsdichte,NaN,NaN,DIE LINKE,158,4.427010
21,NaN,mittlere Besiedlungsdichte,NaN,NaN,FDP,381,10.675259
22,NaN,mittlere Besiedlungsdichte,NaN,NaN,SPD,721,20.201737


In [76]:
#we also merge the datasets of 2021 and 2023 based on all the features for the party preferences
comp_party=pd.merge(df_2021_party,df_2023_party,on=['Alter', 'Besiedelung', 'Geschlecht', 'OstWest', 'pv01'],suffixes=["_2021","_2023"])
comp_party["diff_percent"]=comp_party["percent_2023"]-comp_party["percent_2021"]
comp_party.sort_values("diff_percent")

,Alter,Besiedelung,Geschlecht,OstWest,pv01,absolute_2021,percent_2021,absolute_2023,percent_2023,diff_percent
126,50-64,NaN,NaN,Berlin,SPD,19,29.230769,7,9.459459,-19.771310
157,U30,NaN,NaN,Ost,FDP,28,22.580645,5,3.759398,-18.821247
151,U30,NaN,NaN,Berlin,WUERDE NICHT WAEHLEN,9,18.000000,0,0.000000,-18.000000
170,Ü65,NaN,NaN,Berlin,CDU-CSU,30,35.294118,17,18.279570,-17.014548
146,U30,NaN,NaN,Berlin,CDU-CSU,8,16.000000,0,0.000000,-16.000000
...,...,...,...,...,...,...,...,...,...,...
443,NaN,mittlere Besiedlungsdichte,NaN,Berlin,DIE GRUENEN,0,NaN,0,NaN,NaN
444,NaN,mittlere Besiedlungsdichte,NaN,Berlin,DIE LINKE,0,NaN,0,NaN,NaN
445,NaN,mittlere Besiedlungsdichte,NaN,Berlin,FDP,0,NaN,0,NaN,NaN
446,NaN,mittlere Besiedlungsdichte,NaN,Berlin,SPD,0,NaN,0,NaN,NaN


In [77]:

#and we pivot it based on the party preferences
comp_party_pivot=comp_party.pivot(index=['Alter','Besiedelung','Geschlecht','OstWest'], columns='pv01')[['percent_2021','percent_2023',"absolute_2021","absolute_2023"]].reset_index()
comp_party_pivot

Alter                 Besiedelung Geschlecht OstWest percent_2021  \
pv01                                                               AFD   
0      NaN                         NaN        NaN  Berlin     2.950820   
1      NaN                         NaN        NaN     Ost    11.746522   
2      NaN                         NaN        NaN    West     4.289587   
3      NaN                         NaN   Männlich     NaN     7.808564   
4      NaN                         NaN   Männlich  Berlin     4.827586   
..     ...                         ...        ...     ...          ...   
60     Ü65                         NaN   Männlich     NaN     6.226766   
61     Ü65                         NaN   Weiblich     NaN     1.906275   
62     Ü65             dicht besiedelt        NaN     NaN     3.819918   
63     Ü65            gering besiedelt        NaN     NaN     5.040323   
64     Ü65  mittlere Besiedlungsdichte        NaN     NaN     3.435805   

                                                                 ...  \
pv01 ANDERE PARTEI    CDU-CSU DIE GRUENEN  DIE LINKE        FDP  ...   
0         3.606557  21.639344   26.885246   9.180328   7.213115  ...   
1         5.641422  23.106646   14.992272  12.905719  12.210201  ...   
2         3.412534  25.912933   29.835752   5.469622  11.880083  ...   
3         3.803526  24.055416   24.382872   7.506297  13.173804  ...   
4         2.068966  17.931034   27.586207   7.586207  12.413793  ...   
..             ...        ...         ...        ...        ...  ...   
60        0.743494  30.762082   14.962825   5.947955  13.661710  ...   
61        1.985703  36.219222   17.553614   6.592534  10.484512  ...   
62        1.227831  30.422920   17.462483   4.911323   8.867667  ...   
63        2.016129  35.483871   13.709677   8.266129  11.693548  ...   
64        1.265823  35.081374   16.817360   6.329114  14.104882  ...   

     absolute_2021                      absolute_2023                        \
pv01           SPD WUERDE NICHT WAEHLEN           AFD ANDERE PARTEI CDU-CSU   
0               60                   27            22             6      45   
1              159                   92           294            49     267   
2              961                  243           480           215    1609   
3              618                  147           526           156     940   
4               27                   13            15             3      20   
..             ...                  ...           ...           ...     ...   
60             267                   31            88             8     385   
61             247                   71            57            23     426   
62             202                   42            32             6     229   
63              96                   22            46            13     193   
64             216                   38            67            12     389   

                                                            
pv01 DIE GRUENEN DIE LINKE  FDP   SPD WUERDE NICHT WAEHLEN  
0             84        78   19    60                   23  
1            146       140   77   252                   70  
2           1518       240  641  1389                  318  
3            760       225  442   816                  214  
4             36        41    7    29                   12  
..           ...       ...  ...   ...                  ...  
60           154        54   92   321                   36  
61           219        41   94   388                   52  
62           101        37   84   292                   34  
63            65        24   22   117                   17  
64           207        34   80   300                   37  

[65 rows x 36 columns]

In [78]:
# to remove the double index ot the columns, we bring them to one name in one line
new_columns=[]
for index in comp_party_pivot:
    if index[1]!="":
        new_columns.append(index[1]+"_"+index[0])
    else:
        new_columns.append(index[0])
new_columns
comp_party_pivot.columns=new_columns
comp_party_pivot

,Alter,Besiedelung,Geschlecht,OstWest,AFD_percent_2021,ANDERE PARTEI_percent_2021,CDU-CSU_percent_2021,DIE GRUENEN_percent_2021,DIE LINKE_percent_2021,FDP_percent_2021,...,SPD_absolute_2021,WUERDE NICHT WAEHLEN_absolute_2021,AFD_absolute_2023,ANDERE PARTEI_absolute_2023,CDU-CSU_absolute_2023,DIE GRUENEN_absolute_2023,DIE LINKE_absolute_2023,FDP_absolute_2023,SPD_absolute_2023,WUERDE NICHT WAEHLEN_absolute_2023
0,NaN,NaN,NaN,Berlin,2.950820,3.606557,21.639344,26.885246,9.180328,7.213115,...,60,27,22,6,45,84,78,19,60,23
1,NaN,NaN,NaN,Ost,11.746522,5.641422,23.106646,14.992272,12.905719,12.210201,...,159,92,294,49,267,146,140,77,252,70
2,NaN,NaN,NaN,West,4.289587,3.412534,25.912933,29.835752,5.469622,11.880083,...,961,243,480,215,1609,1518,240,641,1389,318
3,NaN,NaN,Männlich,NaN,7.808564,3.803526,24.055416,24.382872,7.506297,13.173804,...,618,147,526,156,940,760,225,442,816,214
4,NaN,NaN,Männlich,Berlin,4.827586,2.068966,17.931034,27.586207,7.586207,12.413793,...,27,13,15,3,20,36,41,7,29,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,Ü65,NaN,Männlich,NaN,6.226766,0.743494,30.762082,14.962825,5.947955,13.661710,...,267,31,88,8,385,154,54,92,321,36
61,Ü65,NaN,Weiblich,NaN,1.906275,1.985703,36.219222,17.553614,6.592534,10.484512,...,247,71,57,23,426,219,41,94,388,52
62,Ü65,dicht besiedelt,NaN,NaN,3.819918,1.227831,30.422920,17.462483,4.911323,8.867667,...,202,42,32,6,229,101,37,84,292,34
63,Ü65,gering besiedelt,NaN,NaN,5.040323,2.016129,35.483871,13.709677,8.266129,11.693548,...,96,22,46,13,193,65,24,22,117,17


In [79]:
#same is done for inflation
comp_inflation_pivot=comp_inflation.pivot(index=['Alter','Besiedelung','Geschlecht','OstWest'], columns='va03')[['percent_2021','percent_2023',"absolute_2021","absolute_2023"]].reset_index()
new_columns=[]
for index in comp_inflation_pivot:
    if index[1]!="":
        new_columns.append(index[1]+"_"+index[0])
    else:
        new_columns.append(index[0])
new_columns
comp_inflation_pivot.columns=new_columns
comp_inflation_pivot

,Alter,Besiedelung,Geschlecht,OstWest,AM DRITTWICHTIGSTEN_percent_2021,AM VIERTWICHTIGSTEN_percent_2021,AM WICHTIGSTEN_percent_2021,AM ZWEITWICHTIGSTEN_percent_2021,AM DRITTWICHTIGSTEN_percent_2023,AM VIERTWICHTIGSTEN_percent_2023,AM WICHTIGSTEN_percent_2023,AM ZWEITWICHTIGSTEN_percent_2023,AM DRITTWICHTIGSTEN_absolute_2021,AM VIERTWICHTIGSTEN_absolute_2021,AM WICHTIGSTEN_absolute_2021,AM ZWEITWICHTIGSTEN_absolute_2021,AM DRITTWICHTIGSTEN_absolute_2023,AM VIERTWICHTIGSTEN_absolute_2023,AM WICHTIGSTEN_absolute_2023,AM ZWEITWICHTIGSTEN_absolute_2023
0,NaN,NaN,NaN,Berlin,26.296296,48.888889,7.037037,17.777778,34.010152,31.979695,13.451777,20.558376,71,132,19,48,134,126,53,81
1,NaN,NaN,NaN,Ost,28.310502,51.050228,5.388128,15.251142,23.569652,40.858209,11.815920,23.756219,310,559,59,167,379,657,190,382
2,NaN,NaN,NaN,West,25.725726,55.675676,4.924925,13.673674,27.720341,39.738951,11.592143,20.948566,1285,2781,246,683,2145,3075,897,1621
3,NaN,NaN,Männlich,NaN,24.866057,56.760164,5.263158,13.110621,29.421697,41.325454,9.983115,19.269734,789,1801,167,416,1394,1958,473,913
4,NaN,NaN,Männlich,Berlin,19.130435,61.739130,2.608696,16.521739,36.702128,38.297872,10.106383,14.893617,22,71,3,19,69,72,19,28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,Ü65,NaN,Männlich,NaN,21.628190,60.267315,3.766707,14.337789,28.267974,48.447712,4.820261,18.464052,178,496,31,118,346,593,59,226
61,Ü65,NaN,Weiblich,NaN,26.554268,55.742887,1.369863,16.332982,25.371207,48.030988,6.003873,20.593932,252,529,13,155,393,744,93,319
62,Ü65,dicht besiedelt,NaN,NaN,26.262626,51.178451,5.050505,17.508418,28.571429,53.406593,2.527473,15.494505,156,304,30,104,260,486,23,141
63,Ü65,gering besiedelt,NaN,NaN,22.727273,60.962567,0.267380,16.042781,22.203390,47.627119,8.983051,21.186441,85,228,1,60,131,281,53,125


In [80]:
#and then again we merge both datasets
comp_inflation_party_pivot=pd.merge(comp_inflation_pivot,comp_party_pivot,on=['Alter', 'Besiedelung', 'Geschlecht', 'OstWest'])
comp_inflation_party_pivot

,Alter,Besiedelung,Geschlecht,OstWest,AM DRITTWICHTIGSTEN_percent_2021,AM VIERTWICHTIGSTEN_percent_2021,AM WICHTIGSTEN_percent_2021,AM ZWEITWICHTIGSTEN_percent_2021,AM DRITTWICHTIGSTEN_percent_2023,AM VIERTWICHTIGSTEN_percent_2023,...,SPD_absolute_2021,WUERDE NICHT WAEHLEN_absolute_2021,AFD_absolute_2023,ANDERE PARTEI_absolute_2023,CDU-CSU_absolute_2023,DIE GRUENEN_absolute_2023,DIE LINKE_absolute_2023,FDP_absolute_2023,SPD_absolute_2023,WUERDE NICHT WAEHLEN_absolute_2023
0,NaN,NaN,NaN,Berlin,26.296296,48.888889,7.037037,17.777778,34.010152,31.979695,...,60,27,22,6,45,84,78,19,60,23
1,NaN,NaN,NaN,Ost,28.310502,51.050228,5.388128,15.251142,23.569652,40.858209,...,159,92,294,49,267,146,140,77,252,70
2,NaN,NaN,NaN,West,25.725726,55.675676,4.924925,13.673674,27.720341,39.738951,...,961,243,480,215,1609,1518,240,641,1389,318
3,NaN,NaN,Männlich,NaN,24.866057,56.760164,5.263158,13.110621,29.421697,41.325454,...,618,147,526,156,940,760,225,442,816,214
4,NaN,NaN,Männlich,Berlin,19.130435,61.739130,2.608696,16.521739,36.702128,38.297872,...,27,13,15,3,20,36,41,7,29,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,Ü65,NaN,Männlich,NaN,21.628190,60.267315,3.766707,14.337789,28.267974,48.447712,...,267,31,88,8,385,154,54,92,321,36
61,Ü65,NaN,Weiblich,NaN,26.554268,55.742887,1.369863,16.332982,25.371207,48.030988,...,247,71,57,23,426,219,41,94,388,52
62,Ü65,dicht besiedelt,NaN,NaN,26.262626,51.178451,5.050505,17.508418,28.571429,53.406593,...,202,42,32,6,229,101,37,84,292,34
63,Ü65,gering besiedelt,NaN,NaN,22.727273,60.962567,0.267380,16.042781,22.203390,47.627119,...,96,22,46,13,193,65,24,22,117,17


In [81]:
comp_inflation_party_pivot.columns

Index(['Alter', 'Besiedelung', 'Geschlecht', 'OstWest',
       'AM DRITTWICHTIGSTEN_percent_2021', 'AM VIERTWICHTIGSTEN_percent_2021',
       'AM WICHTIGSTEN_percent_2021', 'AM ZWEITWICHTIGSTEN_percent_2021',
       'AM DRITTWICHTIGSTEN_percent_2023', 'AM VIERTWICHTIGSTEN_percent_2023',
       'AM WICHTIGSTEN_percent_2023', 'AM ZWEITWICHTIGSTEN_percent_2023',
       'AM DRITTWICHTIGSTEN_absolute_2021',
       'AM VIERTWICHTIGSTEN_absolute_2021', 'AM WICHTIGSTEN_absolute_2021',
       'AM ZWEITWICHTIGSTEN_absolute_2021',
       'AM DRITTWICHTIGSTEN_absolute_2023',
       'AM VIERTWICHTIGSTEN_absolute_2023', 'AM WICHTIGSTEN_absolute_2023',
       'AM ZWEITWICHTIGSTEN_absolute_2023', 'AFD_percent_2021',
       'ANDERE PARTEI_percent_2021', 'CDU-CSU_percent_2021',
       'DIE GRUENEN_percent_2021', 'DIE LINKE_percent_2021',
       'FDP_percent_2021', 'SPD_percent_2021',
       'WUERDE NICHT WAEHLEN_percent_2021', 'AFD_percent_2023',
       'ANDERE PARTEI_percent_2023', 'CDU-CSU_percent_202

In [82]:
#we take in the data from the zensus that gives a share of the population for each subgroup
#but only with two features defined as the same time
zensus=pd.read_csv("json_none.csv")
zensus

,Geschlecht,Besiedelung,OstWest,Alter,Share
0,NaN,NaN,Ost,U30,1.675804
1,NaN,gering besiedelt,NaN,U30,2.543912
2,NaN,dicht besiedelt,Ost,NaN,3.853220
3,NaN,NaN,Ost,30-49,4.559863
4,NaN,NaN,Ost,50-64,4.751534
5,NaN,NaN,Ost,Ü65,5.482466
6,NaN,gering besiedelt,Ost,NaN,5.491600
7,NaN,gering besiedelt,NaN,30-49,5.723338
8,NaN,mittlere Besiedlungsdichte,NaN,U30,5.842770
9,NaN,dicht besiedelt,NaN,U30,6.005464


In [83]:
# we merge this
comp_inflation_party_pivot_zensus=pd.merge(comp_inflation_party_pivot,zensus,on=['Alter', 'Besiedelung', 'Geschlecht', 'OstWest'])


In [84]:
#and we filter only for the percentage results  
perc_share_df=comp_inflation_party_pivot_zensus[comp_inflation_party_pivot_zensus.columns[~comp_inflation_party_pivot_zensus.columns.str.contains("absolute")]]

In [85]:
perc_share_df

,Alter,Besiedelung,Geschlecht,OstWest,AM DRITTWICHTIGSTEN_percent_2021,AM VIERTWICHTIGSTEN_percent_2021,AM WICHTIGSTEN_percent_2021,AM ZWEITWICHTIGSTEN_percent_2021,AM DRITTWICHTIGSTEN_percent_2023,AM VIERTWICHTIGSTEN_percent_2023,...,WUERDE NICHT WAEHLEN_percent_2021,AFD_percent_2023,ANDERE PARTEI_percent_2023,CDU-CSU_percent_2023,DIE GRUENEN_percent_2023,DIE LINKE_percent_2023,FDP_percent_2023,SPD_percent_2023,WUERDE NICHT WAEHLEN_percent_2023,Share
0,NaN,NaN,NaN,Ost,28.310502,51.050228,5.388128,15.251142,23.569652,40.858209,...,7.109737,22.702703,3.783784,20.617761,11.274131,10.810811,5.945946,19.459459,5.405405,16.469667
1,NaN,NaN,NaN,West,25.725726,55.675676,4.924925,13.673674,27.720341,39.738951,...,3.874980,7.488300,3.354134,25.101404,23.681747,3.744150,10.000000,21.669267,4.960998,79.480028
2,NaN,NaN,Männlich,NaN,24.866057,56.760164,5.263158,13.110621,29.421697,41.325454,...,3.702771,12.895317,3.824467,23.044864,18.632018,5.516058,10.835989,20.004903,5.246384,48.259419
3,NaN,NaN,Männlich,Ost,29.906542,50.841121,5.607477,13.644860,24.871795,44.743590,...,7.184751,27.835052,3.681885,19.882180,9.720177,9.720177,6.627393,15.611193,6.921944,7.956365
4,NaN,NaN,Männlich,West,24.058660,57.788347,5.311138,12.841855,30.000000,40.769231,...,2.704423,9.947482,3.954279,24.250850,20.327464,3.645351,12.048193,21.037998,4.788384,38.393686
5,NaN,NaN,Weiblich,NaN,27.518042,52.431754,4.926263,15.123941,25.269892,37.984806,...,5.512821,6.813020,2.876609,24.753974,24.930608,5.879384,7.443856,22.331567,4.970982,51.683982
6,NaN,NaN,Weiblich,Ost,26.785714,51.250000,5.178571,16.785714,22.342995,37.198068,...,7.026144,17.045455,3.896104,21.428571,12.987013,12.012987,5.194805,23.701299,3.733766,8.513302
7,NaN,NaN,Weiblich,West,27.427184,53.519417,4.530744,14.522654,25.554435,38.760081,...,5.051151,4.979515,2.741885,25.969114,27.103687,3.844942,7.910495,22.313268,5.137094,41.086342
8,NaN,dicht besiedelt,NaN,NaN,27.350427,53.126406,5.937922,13.585245,29.111697,40.662562,...,4.384058,6.645570,2.390999,18.389592,26.722925,7.735584,7.665260,24.964838,5.485232,34.799334
9,NaN,dicht besiedelt,NaN,Ost,36.401674,51.464435,5.020921,7.112971,22.486772,37.830688,...,7.000000,16.993464,3.594771,17.973856,18.627451,10.130719,3.921569,22.875817,5.882353,3.853220


In [86]:
perc_share_df=perc_share_df.round(1)


In [87]:
#and we write this into a csv file
perc_share_df.to_csv("perc_share_df.csv")

In [88]:
# we only use the percentage data
data_with_perc=comp_inflation_party_pivot_zensus[comp_inflation_party_pivot_zensus.columns[~comp_inflation_party_pivot_zensus.columns.str.contains("absolute")]]

In [89]:
data_with_perc=data_with_perc.round(1)
data_with_perc["Geschlecht"]=data_with_perc["Geschlecht"].str.replace("Männlich","Maennlich")
data_with_perc["Alter"]=data_with_perc["Alter"].str.replace("Ü65","65plus")
data_with_perc["Besiedelung"]=data_with_perc["Besiedelung"].str.replace("dicht besiedelt","Stadt")
data_with_perc["Besiedelung"]=data_with_perc["Besiedelung"].str.replace("gering besiedelt","Dorf")
data_with_perc["Besiedelung"]=data_with_perc["Besiedelung"].str.replace("mittlere Besiedlungsdichte","Kleinstadt")

In [90]:
json_array=[]
for index, row in data_with_perc.iterrows():
    print(row)
    base_dict=row[["Alter","Besiedelung","Geschlecht","OstWest","Share"]].to_dict()
    base_dict["inflation"]=row[[ 'AM DRITTWICHTIGSTEN_percent_2021', 'AM VIERTWICHTIGSTEN_percent_2021',
        'AM WICHTIGSTEN_percent_2021', 'AM ZWEITWICHTIGSTEN_percent_2021',
        'AM DRITTWICHTIGSTEN_percent_2023', 'AM VIERTWICHTIGSTEN_percent_2023',
        'AM WICHTIGSTEN_percent_2023', 'AM ZWEITWICHTIGSTEN_percent_2023']].to_dict()
    base_dict["wahl"]=row[['AFD_percent_2021', 'ANDERE PARTEI_percent_2021',
        'CDU-CSU_percent_2021', 'DIE GRUENEN_percent_2021',
        'DIE LINKE_percent_2021', 'FDP_percent_2021', 'SPD_percent_2021',
        'WUERDE NICHT WAEHLEN_percent_2021', 'AFD_percent_2023',
        'ANDERE PARTEI_percent_2023', 'CDU-CSU_percent_2023',
        'DIE GRUENEN_percent_2023', 'DIE LINKE_percent_2023',
        'FDP_percent_2023', 'SPD_percent_2023',
        'WUERDE NICHT WAEHLEN_percent_2023']].to_dict()
    json_array.append(base_dict)
json_array

Alter                                 NaN
Besiedelung                           NaN
Geschlecht                            NaN
OstWest                               Ost
AM DRITTWICHTIGSTEN_percent_2021     28.3
AM VIERTWICHTIGSTEN_percent_2021     51.1
AM WICHTIGSTEN_percent_2021           5.4
AM ZWEITWICHTIGSTEN_percent_2021     15.3
AM DRITTWICHTIGSTEN_percent_2023     23.6
AM VIERTWICHTIGSTEN_percent_2023     40.9
AM WICHTIGSTEN_percent_2023          11.8
AM ZWEITWICHTIGSTEN_percent_2023     23.8
AFD_percent_2021                     11.7
ANDERE PARTEI_percent_2021            5.6
CDU-CSU_percent_2021                 23.1
DIE GRUENEN_percent_2021             15.0
DIE LINKE_percent_2021               12.9
FDP_percent_2021                     12.2
SPD_percent_2021                     12.3
WUERDE NICHT WAEHLEN_percent_2021     7.1
AFD_percent_2023                     22.7
ANDERE PARTEI_percent_2023            3.8
CDU-CSU_percent_2023                 20.6
DIE GRUENEN_percent_2023          

[{'Alter': nan,
  'Besiedelung': nan,
  'Geschlecht': nan,
  'OstWest': 'Ost',
  'Share': 16.5,
  'inflation': {'AM DRITTWICHTIGSTEN_percent_2021': 28.3,
   'AM VIERTWICHTIGSTEN_percent_2021': 51.1,
   'AM WICHTIGSTEN_percent_2021': 5.4,
   'AM ZWEITWICHTIGSTEN_percent_2021': 15.3,
   'AM DRITTWICHTIGSTEN_percent_2023': 23.6,
   'AM VIERTWICHTIGSTEN_percent_2023': 40.9,
   'AM WICHTIGSTEN_percent_2023': 11.8,
   'AM ZWEITWICHTIGSTEN_percent_2023': 23.8},
  'wahl': {'AFD_percent_2021': 11.7,
   'ANDERE PARTEI_percent_2021': 5.6,
   'CDU-CSU_percent_2021': 23.1,
   'DIE GRUENEN_percent_2021': 15.0,
   'DIE LINKE_percent_2021': 12.9,
   'FDP_percent_2021': 12.2,
   'SPD_percent_2021': 12.3,
   'WUERDE NICHT WAEHLEN_percent_2021': 7.1,
   'AFD_percent_2023': 22.7,
   'ANDERE PARTEI_percent_2023': 3.8,
   'CDU-CSU_percent_2023': 20.6,
   'DIE GRUENEN_percent_2023': 11.3,
   'DIE LINKE_percent_2023': 10.8,
   'FDP_percent_2023': 5.9,
   'SPD_percent_2023': 19.5,
   'WUERDE NICHT WAEHLEN_perc

In [91]:
import json

with open('data_for_viz_2.json', 'w', encoding='utf-8') as f:
    json.dump(json_array, f)